In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env

In [ ]:
# Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips 
# from the zillow database for all 'Single Family Residential' properties.

In [3]:
# Read data from the properties_2017 table in the zillow database on our mySQL server. 

import os

def get_db_url(db, username=env.username, host=env.host, password=env.password):
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [4]:
def get_zillow_data(filename, query, db):

    if os.path.isfile(filename):

        return pd.read_csv(filename)
    else:
        # Create the url
        url = get_db_url(db)

        # Read the SQL query into a dataframe
        df = pd.read_sql(query, url)

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df

In [5]:
filename = "zillow.csv"
db = "Zillow"
query = '''SELECT bathroomcnt, 
           calculatedfinishedsquarefeet, 
           yearbuilt, 
           taxamount, 
           fips, 
           taxvaluedollarcnt
           FROM properties_2017'''

In [7]:
df = get_zillow_data(filename, query, db)

In [8]:
df.head()

,Unnamed: 0,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxamount,fips,taxvaluedollarcnt
0,0,0.0,0.0,NaN,NaN,NaN,6037.0,27516.0
1,1,0.0,0.0,NaN,NaN,NaN,6037.0,10.0
2,2,0.0,0.0,NaN,NaN,NaN,6037.0,10.0
3,3,0.0,0.0,NaN,NaN,174.21,6037.0,2108.0
4,4,2.0,4.0,3633.0,2005.0,6941.39,6037.0,296425.0


In [ ]:
# Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb 
# file like we did above. You may handle the missing values however you feel is appropriate and meaningful; 
# remember to document your process and decisions using markdown and code commenting where helpful.

In [9]:
    # a. drop null values:
df.isna().sum()

Unnamed: 0                         0
bathroomcnt                       11
bedroomcnt                        11
calculatedfinishedsquarefeet    8484
yearbuilt                       9337
taxamount                       4442
fips                               0
taxvaluedollarcnt                493
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Unnamed: 0                    2140235 non-null  int64  
 1   bathroomcnt                   2140235 non-null  float64
 2   bedroomcnt                    2140235 non-null  float64
 3   calculatedfinishedsquarefeet  2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
 7   taxvaluedollarcnt             2140235 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 147.0 MB


In [ ]:
    # b. check data types

In [12]:
df.dtypes

Unnamed: 0                        int64
bathroomcnt                     float64
bedroomcnt                      float64
calculatedfinishedsquarefeet    float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
taxvaluedollarcnt               float64
dtype: object

In [15]:
df.drop(columns='Unnamed: 0')

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxamount,fips,taxvaluedollarcnt
4,2.0,4.0,3633.0,2005.0,6941.39,6037.0,296425.0
6,4.0,3.0,1620.0,2011.0,10244.94,6037.0,847770.0
7,2.0,3.0,2077.0,1926.0,7924.68,6037.0,646760.0
11,0.0,0.0,1200.0,1972.0,91.60,6037.0,5328.0
14,0.0,0.0,171.0,1973.0,255.17,6037.0,6920.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,2015.0,13877.56,6037.0,422400.0
2152858,3.0,4.0,2262.0,2015.0,13494.52,6059.0,960756.0
2152859,4.5,4.0,3127.0,2014.0,6244.16,6059.0,536061.0
2152861,2.5,3.0,1974.0,2015.0,5302.70,6059.0,424353.0


In [16]:
df.dtypes

Unnamed: 0                        int64
bathroomcnt                     float64
bedroomcnt                      float64
calculatedfinishedsquarefeet    float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
taxvaluedollarcnt               float64
dtype: object

In [19]:
(df['fips'] != df['fips'].astype(int)).sum()
cast_to_int = []
for col in df.columns:
    sum_of_discord = (df[col] != df[col].astype(int)).sum()
    if sum_of_discord > 0:
        print(f'{col} has data that is probably not interpretable as an int! It has {sum_of_discord / df.shape[0] * 100} percent mismatched values!')
    else:
        print(f'{col} is good to go for int casting!')
        cast_to_int.append(col)
    print('============')

Unnamed: 0 is good to go for int casting!
bathroomcnt has data that is probably not interpretable as an int! It has 10.748072057507702 percent mismatched values!
bedroomcnt is good to go for int casting!
calculatedfinishedsquarefeet is good to go for int casting!
yearbuilt is good to go for int casting!
taxamount has data that is probably not interpretable as an int! It has 98.66299728768102 percent mismatched values!
fips is good to go for int casting!
taxvaluedollarcnt is good to go for int casting!


In [20]:
cast_to_int

['Unnamed: 0',
 'bedroomcnt',
 'calculatedfinishedsquarefeet',
 'yearbuilt',
 'fips',
 'taxvaluedollarcnt']

In [21]:
for col in cast_to_int:
    df.loc[:,col] = df[col].astype(int)

In [22]:
    # c. Rename columns
df.columns
df = df.rename(columns={
    'bedroomcnt': 'bedrooms',
    'bathroomcnt': 'bathrooms',
    'calculatedfinishedsquarefeet': 'sqft',
    'taxvaluedollarcnt': 'taxvalue',
    'fips': 'county'
})

In [26]:
county_maps = {6037: 'LA',
6059: 'Orange',
6111: 'Ventura'
}

In [27]:
df['county'] = df.county.map(county_maps)

In [ ]:
# Write a function to split your data into train, validate, and test.

In [28]:
def split_zillow_data(df):
    '''
    take in a DataFrame and return train, validate, test DataFrames.
    '''

    # splits df into train_validate and test using train_test_split() stratifying on species to get an even mix of each species
    train_validate, test = train_test_split(df, test_size=.2, random_state=1349)

    # splits train_validate into train and validate using train_test_split() stratifying on species to get an even mix of each species
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=1349)
    return train, validate, test

In [ ]:
# Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned 
# dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [29]:
def get_zillow_data():
    '''
    This function acquires zillow.csv it is available
    otherwise, it makes the SQL connection and uses the query provided
    to read in the dataframe from SQL.
    If they csv is not present, it will write one.
    '''
    filename = "zillow_2017.csv"

    if os.path.isfile(filename):

        return pd.read_csv(filename, index_col=0)
    else:
        # Create the url
        db = 'zillow'
        url = f'mysql+pymysql://{user}:{password}@{host}/{db}'
        
        sql_query = '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
            FROM properties_2017
            WHERE propertylandusetypeid = 261'''

        # Read the SQL query into a dataframe
        df = pd.read_sql(zillow_query, url)

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df

In [30]:
def prep_zillow(df):
    '''
    This function takes in a dataframe
    renames the columns and drops nulls values
    Additionally it changes datatypes for appropriate columns
    and renames fips to actual county names.
    Then returns a cleaned dataframe
    '''
    df = df.rename(columns = {'bedroomcnt':'bedrooms',
                     'bathroomcnt':'bathrooms',
                     'calculatedfinishedsquarefeet':'area',
                     'taxvaluedollarcnt':'taxvalue',
                     'fips':'county'})
    
    df = df.dropna()
    
    make_ints = ['bedrooms','area','taxvalue','yearbuilt']

    for col in make_ints:
        df[col] = df[col].astype(int)
        
    df.county = df.county.map({6037:'LA',6059:'Orange',6111:'Ventura'})
    
    return df